# Classification Census income

In [65]:
# import the required classed and packages

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# Data Ingestion

In [2]:
import pandas as pd
data=pd.read_csv(r"Adult.csv")

In [3]:
data = data.sample(15000)

# EDA

In [4]:
data.shape

(15000, 15)

In [97]:
data.head(2)

,age,workclass,fnlwgt,education,education-num,marital_status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native_country,income
27288,24,Private,139989,Bachelors,13,Never-married,Sales,Own-child,Black,Male,0,0,40,United-States,0
25341,41,Local-gov,227644,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0


In [98]:
data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native_country',
       'income'],
      dtype='object')

In [99]:
data.rename(columns = {'marital-status':'marital_status', 'native-country':'native_country'}, inplace = True)

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 27288 to 21816
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             15000 non-null  int64 
 1   workclass       15000 non-null  object
 2   fnlwgt          15000 non-null  int64 
 3   education       15000 non-null  object
 4   education-num   15000 non-null  int64 
 5   marital_status  15000 non-null  object
 6   occupation      15000 non-null  object
 7   relationship    15000 non-null  object
 8   race            15000 non-null  object
 9   sex             15000 non-null  object
 10  capital-gain    15000 non-null  int64 
 11  capital-loss    15000 non-null  int64 
 12  hours-per-week  15000 non-null  int64 
 13  native_country  15000 non-null  object
 14  income          15000 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 1.8+ MB


In [101]:
data.shape

(15000, 15)

In [102]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,15000.0,38.662600,13.655587,17.0,28.00,37.0,48.0,90.0
fnlwgt,15000.0,189413.755267,104412.147101,13769.0,118354.75,178369.5,236436.0,1268339.0
education-num,15000.0,10.082867,2.582230,1.0,9.00,10.0,12.0,16.0
capital-gain,15000.0,1064.732667,7329.293987,0.0,0.00,0.0,0.0,99999.0
capital-loss,15000.0,84.144467,393.405456,0.0,0.00,0.0,0.0,3900.0
hours-per-week,15000.0,40.375133,12.355827,1.0,40.00,40.0,45.0,99.0
income,15000.0,0.237800,0.425750,0.0,0.00,0.0,0.0,1.0


In [140]:
data['income'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

In [141]:
data["income"] = data["income"].astype("float")

In [142]:
# define numerical & categorical columns
numeric_features = [feature for feature in data.columns if data[feature].dtype != 'O']
categorical_features = [feature for feature in data.columns if data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 7 numerical features : ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income']

We have 8 categorical features : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']


In [143]:
data.groupby(['workclass']).size()

workclass
?                     879
Federal-gov           426
Local-gov             948
Never-worked            3
Private             10401
Self-emp-inc          502
Self-emp-not-inc     1220
State-gov             614
Without-pay             7
dtype: int64

In [144]:
list1 = data.columns.tolist()
for i in list1:
    print(data.groupby([i]).size().sort_values(ascending=False))
    print("**********************")

age
33    421
36    417
23    401
34    401
41    392
     ... 
82      5
83      3
88      2
85      1
86      1
Length: 72, dtype: int64
**********************
workclass
Private             10401
Self-emp-not-inc     1220
Local-gov             948
?                     879
State-gov             614
Self-emp-inc          502
Federal-gov           426
Without-pay             7
Never-worked            3
dtype: int64
**********************
fnlwgt
188246     9
125892     7
121124     7
172538     6
113364     6
          ..
152636     1
152657     1
152683     1
152696     1
1268339    1
Length: 11927, dtype: int64
**********************
education
HS-grad         4855
Some-college    3349
Bachelors       2466
Masters          816
Assoc-voc        657
11th             502
Assoc-acdm       474
10th             430
7th-8th          302
Prof-school      265
9th              220
12th             206
Doctorate        185
5th-6th          158
1st-4th           87
Preschool         28
dtype: int6

In [145]:
# Analysis

Most of the population in the age group 30 to 40

2/3 of the population works in the private sector

Most of the population is HS-grad, Some-college, Bachelors

80% of the population is white, 10% is black, rest are from other races

Male population is twice the population of female

Majority of the population data (90%) is from the US

# Preprocessing

In [146]:
# Check for null values in the data

In [147]:
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native_country    0
income            0
dtype: int64

the data has no null values

In [148]:
# Let us group by occupation

In [149]:
data.groupby('native_country').size().sort_values(ascending=False)

native_country
United-States                 13463
Mexico                          316
?                               254
Philippines                      98
Germany                          63
Canada                           54
Cuba                             46
El-Salvador                      46
Puerto-Rico                      44
India                            44
England                          38
Jamaica                          37
Italy                            35
China                            35
Guatemala                        34
South                            34
Vietnam                          33
Columbia                         29
Dominican-Republic               25
Japan                            24
Poland                           21
Taiwan                           20
Ecuador                          18
Iran                             17
Greece                           17
Peru                             17
Portugal                         15
Haiti        

USA has the largest data, followed by mexico, ?, phillipplines

Hungary has the least data in this dataset

In [150]:
# Now let us group by income 

In [151]:
data.groupby('income').size().sort_values(ascending=False)

income
0.0    11433
1.0     3567
dtype: int64

Largest population earns less than or equal to 50k per annum

close to quarter of the population earns more than 50k per annum

In [152]:
# Now let us group by sex

In [153]:
data.groupby('sex').size().sort_values(ascending=False)

sex
Male      9984
Female    5016
dtype: int64

Male population is twice the population of Female population in the dataset

# Splitting of the Data

In [154]:
y = data['income']
X = data.drop('income', axis = 1)

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.2) 

In [156]:
category_var = [col for col in X.columns if X[col].dtypes == object]
category_var

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country']

In [157]:
numeric_var = [col for col in X.columns if X[col].dtypes != object]
numeric_var

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [158]:
# Feature Scaling

In [159]:
one_hot = ce.OneHotEncoder(cols = category_var, handle_unknown = 'ignore')

# Creating dataframe for categorical variables which converted to one hot encoded variables.
X_train_one_hot = pd.DataFrame(one_hot.fit_transform(X_train))
X_test_one_hot = pd.DataFrame(one_hot.transform(X_test))


X_train_one_hot.index = X_train.index
X_test_one_hot.index = X_test.index

num_X_train = X_train[numeric_var]
num_X_test = X_test[numeric_var]

# Joining numerical and one hot encoded variables to create our final X_train and X_test.
X_train_new = pd.concat([num_X_train, X_train_one_hot], axis = 1)
X_test_new = pd.concat([num_X_test, X_test_one_hot], axis = 1)

In [160]:
# Scaling our records into standard range of 0 and 1.
scaler = StandardScaler()

X_train_new = scaler.fit_transform(X_train_new)
X_test_new = scaler.transform(X_test_new)

# Logistic Regression

In [161]:
model_logr = LogisticRegression(random_state = 1,solver="liblinear")
model_logr.fit(X_train_new, y_train)
pred_logr = model_logr.predict(X_test_new)
print('Logistic Regression accuracy score:{0:0.2f}%'. format(accuracy_score(y_test, pred_logr)*100))

Logistic Regression accuracy score:85.70%


# SVC

In [162]:
classifier = SVC(random_state = 0,kernel = 'rbf')
classifier.fit(X_train_new, y_train)

SVC(random_state=0)

In [127]:
#Predicting the Test Set

In [128]:
y_pred = classifier.predict(X_test_new)

In [129]:
accuracy_score(y_test, y_pred)

0.8453333333333334

In [174]:
y_pred.shape

(3000,)

In [130]:
# Now let us apply the kernel functions 

In [131]:
classifier = SVC(kernel="linear")
classifier.fit(X_train_new, y_train)
y_pred = classifier.predict(X_test_new)
accuracy_score(y_test, y_pred)

0.8513333333333334

In [132]:
classifier = SVC(kernel="poly")
classifier.fit(X_train_new, y_train)
y_pred = classifier.predict(X_test_new)
accuracy_score(y_test, y_pred)

0.829

In [133]:
classifier = SVC(kernel="rbf")
classifier.fit(X_train_new, y_train)
y_pred = classifier.predict(X_test_new)
accuracy_score(y_test, y_pred)

0.8453333333333334

In [134]:
classifier = SVC(kernel="sigmoid")
classifier.fit(X_train_new, y_train)
y_pred = classifier.predict(X_test_new)
accuracy_score(y_test, y_pred)

0.8406666666666667

SVC with kernel = sigmoid / linear gives the highest accuracy

In [135]:
model = SVC(kernel="sigmoid")
model.fit(X_train_new, y_train)

SVC(kernel='sigmoid')

# Evaluation of the model

In [163]:
from sklearn import metrics

In [164]:
from sklearn.metrics import roc_auc_score

In [179]:
from sklearn.metrics import r2_score

In [70]:
# Classification confusion matrix, ROC AUC score

In [166]:
confusion_matrix(y_test, y_pred)

array([[2104,  192],
       [ 286,  418]], dtype=int64)

In [167]:
# r2 score 

In [176]:
metrics.r2_score(y_test, y_pred)

0.11283457396262275

In [ ]:
# Adjusted R square

In [180]:
metrics.roc_auc_score(y_test, y_pred)

0.7550631533101045

# Save the model using pickle and then use pickle to predict

In [50]:
import pickle

In [45]:
file = open('power_svr.pkl', 'wb')

In [ ]:
# dump file model info file
pickle.dump(model, file)

In [ ]:
file.close()  # close the file

In [46]:
model = open('power_svr.pkl', 'rb')
power = pickle.load(model)

In [51]:
# Predict using the pickle

In [52]:
y_predict = power.predict(X_test_new)

In [53]:
y_predict

array(['>50K', '<=50K', '<=50K', ..., '<=50K', '<=50K', '<=50K'],
      dtype=object)